# OpenDataScience questionnaire EDA

Data visualization is a very interesting subject, very creative and subjective. But there are some common principles that drive the work of every DataScientists, and the most important are:
- Clarity
- Interpretability

We will try to present some common techniques to represent data, more over we will try to show how different types of plots or data manipulation can make a plot more interpretable

# Data preparation

In [1]:
!pip install matplotlib wordcloud plotly 
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

from wordcloud import WordCloud

df = pd.read_csv("2020_opendatascience_poll_data.csv")

In [2]:
df.head()

,Timestamp,What is your residence country (where are you from?),Your timezone (we need that to schedule post timing better),Education,Work status,Data Science expertise level,Age,Are you satisfied with channel update frequency?,Are you satisfied with channel's material complexity?,What field are you interested in (multiple choices are possible),How did you find out about the channel?,How likely are you going to recommend a channel to your friend or colleague?,What’s the main reason for your score? *,"If you want to reach for the editors and to write something, please use the field below:"
0,2020/01/29 1:30:57 PM GMT+3,Ukraine,GMT+3,Undergrad,Student + part time remote job,Middle,18-24,"Yes, it's about perfect",It's all ok,#CV #DL #imageprocessing #videolearning;#RL #D...,Forward from a friend,5,All stuff is absolutely brilliant! Thank you f...,NaN
1,2020/01/29 1:31:19 PM GMT+3,Russia,GMT+3,Graduate,Employed,Middle,31-42,"Nope, less frequent posting will be all right ...",It's all ok,#RL #DL;#NLP #NLU #conversational #dialoguesys...,"It's been so long time ago, I can't remember (...",4,it's ok,"post some jobs with salary ranges, especially ..."
2,2020/01/29 1:32:48 PM GMT+3,Ukraine,GMT+2,PhD,Unemployed,"Novice (Studying courses, active learning)",25-30,"Yes, it's about perfect",Need more specific and complicated materials,#WhereToStart #EntryLevel #Novice #MOOC #Learn...,Telegram channel search,3,NaN,NaN
3,2020/01/29 1:33:27 PM GMT+3,Italy,GMT+1,"No degree at all, still learning / self-taught",Student,"Novice (Studying courses, active learning)",18-24,"Yes, it's about perfect",It's all ok,#CV #DL #imageprocessing #videolearning;#RL #D...,Forward from a friend,5,Mainly due to material shared,NaN
4,2020/01/29 1:33:49 PM GMT+3,Ukraine,GMT+2,Graduate,Employed,Middle,18-24,"Yes, it's about perfect",It's all ok,#RL #DL;#NLP #NLU #conversational #dialoguesys...,"It's been so long time ago, I can't remember (...",2,It's not super useful actually. Good enough to...,NaN


In [3]:
df.columns

Index(['Timestamp', 'What is your residence country (where are you from?)',
       'Your timezone (we need that to schedule post timing better)',
       'Education', 'Work status', 'Data Science expertise  level', 'Age',
       'Are you satisfied with channel update frequency?',
       'Are you satisfied with channel's material complexity?',
       'What field are you interested in (multiple choices are possible)',
       'How did you find out about the channel?',
       'How likely are you going to recommend a channel to your friend or colleague?',
       'What’s the main reason for your score? *',
       'If you want to reach for the editors and to write something, please use the field below:'],
      dtype='object')

In order to manipulate the dataset easily we will change names of columns. So we will work with shorter names.

In [4]:
mapper = {
    "Your timezone (we need that to schedule post timing better)": "Timezone",
    "Work status": "Work",
    "What is your residence country (where are you from?)": "Country",
    "Data Science expertise  level": "Experience",
    "Are you satisfied with channel update frequency?": "Sat_update",
    "Are you satisfied with channel's material complexity?": "Sat_material",
    "What field are you interested in (multiple choices are possible)": "Interests",
    "How did you find out about the channel?": "How_found",
    "How likely are you going to recommend a channel to your friend or colleague?": "Recommend",
    "What’s the main reason for your score? *": "Why",
}

df = df.rename(columns=mapper)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 557 entries, 0 to 556
Data columns (total 14 columns):
 #   Column                                                                                    Non-Null Count  Dtype 
---  ------                                                                                    --------------  ----- 
 0   Timestamp                                                                                 557 non-null    object
 1   Country                                                                                   548 non-null    object
 2   Timezone                                                                                  557 non-null    object
 3   Education                                                                                 557 non-null    object
 4   Work                                                                                      557 non-null    object
 5   Experience                                                      

In [6]:
df.Education.unique()

array(['Undergrad', 'Graduate', 'PhD',
       'No degree at all, still learning / self-taught',
       "I hold a degree in a field irrelevant to DS but as it comes to DS, I'm all self-taught",
       'Post-Doc'], dtype=object)

Now we are ready to perform our EDA. Let's go.

# 01 Countries

Let's start to see where the audience come from. We aggregate data using the value_counts() operation provided by pandas.
We obtain a DataFrame that will be fed to plotly to create the plot. Plotly works in this way:
- Provide the DataFrame
- Provide columns of the DataFrame that will be plotted

(Actually Plotly can work also with standard arrays, but here we will use always DataFrames)

In [7]:
countries = df["Country"].value_counts().to_frame()
print(countries.head()) #here names of Country are the index of the DataFrame
countries = countries.reset_index()
print(countries.head()) #here the index became an independent column and a new numeric index is created
countries = countries.rename(columns={"Country":"Count","index":"Country"}) #simply rename the column for clarity
print(countries.head())

         Country
Russia       154
India        110
Ukraine       45
Spain         30
Italy         29
     index  Country
0   Russia      154
1    India      110
2  Ukraine       45
3    Spain       30
4    Italy       29
   Country  Count
0   Russia    154
1    India    110
2  Ukraine     45
3    Spain     30
4    Italy     29


Now we are ready to use APIs of plotly to create fantastic plots.
Let's start with one of the most common charts, the Pie Chart

In [ ]:
fig = px.pie(countries, values='Count', names='Country', title='Countries of the audience')
fig.show("svg")

It contains all information we were looking for, but there is a problem, isn't it?
Due the great fragmentation of countries, It's difficult to see where your country is (in particular if It has few interviewed).
Let's try to fix this using another plot, the Bar Plot.

Note the 'fig.show("svg")': Plotly by default displays interactive plots. But to correctly display these plots on Github we need to create static plots. If you run this notebook try to remove "svg" and see what happen.

In [ ]:
countries = countries.sort_values(by="Country", ascending=False) #sort value by country name
fig = px.bar(countries, x='Count', y='Country', title='Countries of the audience', 
             hover_data=['Count'], color='Count', orientation='h')
fig.update_layout(
    yaxis_tickfont_size=10
)
fig.show("svg")

With this visualization info should be slightly easier to be retrieved. But the plot is very sparse and not very beautiful. We can do better, for example choosing only the K-heighest countries.

In [ ]:
K = 18
countries_to_plot = countries.sort_values(by="Count", ascending=False)[K::-1] #select K countries with more responses
fig = px.bar(countries_to_plot, x='Count', y='Country', title='Countries of the audience', 
             hover_data=['Count'], color='Count', orientation='h')
fig.update_layout(
    yaxis_tickfont_size=10
)
fig.show("svg")

# 02 Work status

Let's then analyze the people's work status.
We process data like we did previously.

In [ ]:
works = df["Work"].value_counts().to_frame()
print(works.head()) 
works = works.reset_index()
print(works.head()) 
works = works.rename(columns={"Work":"Count","index":"Work"}) #simply rename the column for clarity
print(works.head())

In [ ]:
works = works.sort_values(by="Count", ascending=True)
fig = px.bar(works, x='Count', y='Work',  title='Work status of the audience', 
             hover_data=['Count'], color='Count', orientation='h')
fig.update_layout(
    yaxis_tickfont_size=10
)

fig.show("svg")

<a name="student"></a>
# 03 Countries and Work Status together

Now that we have have an idea of where people are from and which is their work status, let's try to combine these 2 information.
Our objective is now to represent the relation between each country and the status work of the audience.
To accomplish this we will use the [Violin Plot](https://towardsdatascience.com/violin-plots-explained-fb1d115e023d)
We will plot only the top 5 countries.

In [ ]:
K = 5
top_K_countries = countries_to_plot["Country"][-K:]
df_top_countries = df.loc[df['Country'].isin(top_K_countries)].copy()

Before creating the plot we change the original possible responses into a simplified and shorter version, in order to enable the labels to better fit the plot.

In [ ]:
mapper = {
    "Employed remotely": "Employed remotely",
    "Self-employed (freelance)" : "Freelencer",
    "Student + part time job": "Worker student",
    "Self-employed (co-founder / owner)":"Self-employed",
    "Student": "Student",
    "Unemployed": "Unemployed",
    "Employed": "Employed",
    "Student + part time remote job": "Remote Worker student"
}
df_top_countries["Work"] = df_top_countries["Work"].map(mapper)

fig = px.violin(df_top_countries, y="Work", x="Country", title="Relationship between country and work status",
          hover_data=df.columns)
fig.show("svg")

So we can conclude that generally the distribution is quite uniform across different countries, except for India where the number of student is slightly greater then the others top 5 countries.

<a name="age"></a>
# 04 Age

We can adopt the Violin Plot also to represent the age distribution of the audience among countries.

In [ ]:
df_top_countries_by_age = df_top_countries.sort_values(by="Age", ascending=True)
fig = px.violin(df_top_countries_by_age, y="Age", x="Country", title="Relationship between countries and age",
                hover_data=df.columns)
fig.show("svg")

# 05 Is the audience satified about the material and update frequency?

One of the main reasons behind the survey is to understand if the audience is satified about the complexity of the material.
To better understand if the audience is satisfied we can (again) use a Violin Plot.

In [ ]:
mapper = {
    "Need more beginners' stuff": "Too complex",
    "Need more specific and complicated materials" : "Too simple",
    "It's all ok": "Perfect"
}

df["Sat_material"] = df["Sat_material"].map(mapper)
fig = px.violin(df, y="Sat_material", x="Work",hover_data=df.columns,
                title='Distribution of satisfaction for material complexity')
fig.show("svg")

Let's do the same for the update frequency.

In [ ]:
mapper = {
    "Nope, need more information": "Want more",
    "Nope, less frequent posting will be all right by me" : "Want less",
    "Yes, it's about perfect": "Perfect"
}

df["Sat_update"] = df["Sat_update"].map(mapper)
fig = px.violin(df, y="Sat_update", x="Work",hover_data=df.columns, 
                title='Distribution of satisfaction for update frequency')
fig.show("svg")

In this case the plot is meaningless, thus another perspective is required.
Let's try with a Bar Plot.

In [ ]:
update_freq = pd.DataFrame(df.groupby("Sat_update").size(), columns=["Count"]).reset_index().sort_values(by="Count")
fig = px.bar(update_freq, x='Count', y='Sat_update',  title='Distribution of satisfaction for update frequency', 
             hover_data=['Count'], color='Count', orientation='h')
fig.update_layout(
    yaxis_tickfont_size=10
)
fig.show("svg")

# 06 Interests

Given the people interests, create a WordCloud to see trends of topics

In [ ]:
def display_wordcloud_image(text:str):
    """Function for displaying wordcloud of the provided text."""
    plt.figure(figsize = (12, 8))
    wordcloud = WordCloud(max_font_size=None, background_color='white', collocations=True,
                          width=1200, height=1000).generate(text)
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()

In [ ]:
text = ' '.join(df['Interests'].values)
display_wordcloud_image(text)

We could also see if particular trends occures in different countries. We could use a WordCloud for each countries or we could simply select the top-3 topics for some countries. Let's try with the hardest way,the latter one.

First let's select a subset of countries to plot like we did before. We'll keep the top-4 countries.

In [ ]:
K = 4
top_K_countries = countries_to_plot["Country"][-K:]
df_top_countries = df.loc[df['Country'].isin(top_K_countries)].copy()

In order to organize the plot into 4 different subplots (one for each country) we need to use graph_objects and make_subplots.
After importing necessary packages we create a grouby object. It cointains all rows of the dataframe but divided into groups. Each group contains the name (that is the value we are grouping by, in out case Country) and as many rows as submissions for that country.

In [ ]:
from collections import Counter
import plotly.graph_objects as go
from plotly.subplots import make_subplots

g = df_top_countries.groupby("Country")

We then extract names of countries as they appear in the group (thus we can't simply do df_top_countries.unique() because It would returns the same names but with a possibly different order).
These names are used as title for each subplot in order to associate the Pie chart with the country It represents.

In [ ]:
titles = tuple(i for i in g.apply(lambda x: x.name ))

specs = [[{'type':'domain'}, {'type':'domain'}], [{'type':'domain'}, {'type':'domain'}]]
fig = make_subplots(rows=2, cols=2, specs=specs, subplot_titles=titles) #we create a grid of 2x2. Each cell will contain a plot

Then we iterate over all groups and we add one Pie chart at the time

In [ ]:
r , c = 1 , 1 #stands for row and column
for index,sample in g: #iterate over groups 
    counter = Counter() #counts occurences of words over all countries
    
    for index2,row in sample.iterrows(): #iterate over rows of the current country
        tmp = Counter(row["Interests"].split()) #here we count occurences of words for this single row
        counter = counter + tmp #sums single countings together
        
    most_common = counter.most_common(3) #will looks like: [('#WhereToStart', 81), ('#EntryLevel', 81), ('#Novice', 81)]
    
    labels = [e[0] for e in most_common] #extract the first element, the label
    values = [e[1] for e in most_common] #extract the second element, the value
    
    fig.add_trace(go.Pie(labels=labels, values=values),r,c) #Define pie chart subplot
    c = c + 1 #to place correcly each subplot into the 2x2 grid
    if c % 3 == 0:
        r = r + 1
        c = 1
    
# Tune layout
fig.update(layout_title_text='Distribution of top 3 topics in the 4 top countries',
           layout_showlegend=True)
fig = go.Figure(fig)
fig.show("svg")

From this plot we can see that DeepLearning and image processing are very popular across countries, while in India people demand for beginners stuff.
This can be derived by the fact that, as we saw in **03** and **04** India has the highest percentage of student and people ranging from 18 to 24 years old.